In [ ]:
"""
Transform probability profiles via Symbolic Aggregate Approximation
"""

import pandas as pd
import plotly.express as px
import pyts

In [ ]:
from examples.LoadProfileGenerator import example_lpg_validation as example

In [ ]:
input, validation = example.validate_lpg()
prob_v = validation.probability_profiles
prob_i = input.probability_profiles

In [ ]:
def get_sax_mean_curve(data, n_bins = 5):
    # reshape (requirement for SAX)
    d = data.values.reshape(1, -1)
    sax = pyts.approximation.SymbolicAggregateApproximation(n_bins, strategy="uniform")
    sax_array = sax.fit_transform(d)
    d_sax = pd.Series(sax_array[0], name="s", index=data.index)
    # get all symbols
    symbols = d_sax.unique()
    # create a series that only contains the means for each symbol
    means = {s: data[d_sax == s].mean() for s in symbols}
    mean_curve = d_sax.map(means)
    return mean_curve

In [ ]:
category = "cook"
n_bins = 4
mean_curve = get_sax_mean_curve(prob_v.loc[category], n_bins)
mean_curve.name = "Validataion means"
mean_curve_in = get_sax_mean_curve(prob_i.loc[category], n_bins)
mean_curve_in.name ="Input means"
result = pd.concat([mean_curve, mean_curve_in], axis=1)

In [ ]:
dv = prob_v.loc[category]
dv.name = "Validation"
di = prob_i.loc[category]
di.name = "Input"
orig = pd.concat([dv, di], axis=1)
combined = pd.concat([result, orig], axis=1)
fig = px.line(combined)
fig.show()